In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.vgg import vgg13
import torchvision.transforms as T

In [2]:
vggNet=vgg13(pretrained=True)

In [13]:
features=vggNet.features
features=nn.Sequential(*list(features.children())[:-1])
classifer=vggNet.classifier

In [22]:
for i in list(features.children())[:9]:
    i.requires_grad=False
    print(i)

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace)


In [5]:
class RPN_TEST(nn.Module):
    def __init__(self, in_channels = 512, mid_channels = 512):
        super().__init__()
        self.feature=features
        self.conv1=nn.Conv2d(in_channels,mid_channels, kernel_size=3, stride=1, padding=1)
        self.score=nn.Conv2d(mid_channels, 2*9, kernel_size=1, stride=1, padding=0)
        self.bbox =nn.Conv2d(mid_channels, 4*9, kernel_size=1, stride=1, padding=0)
        
    def forward(self, img):
        feature=self.feature(img)
        feature=self.conv1(feature)
        score=self.score(feature)
        bbox=self.bbox(feature)
        return score, bbox


In [6]:
rNet=RPN_TEST()

In [7]:
import numpy as np
from skimage import io
import matplotlib.pyplot as plt

%matplotlib inline

In [8]:
img=io.imread('demo_2.jpg')

In [9]:
USE_GPU=True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

device=torch.device('cuda')
print(device)

cuda


In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

t_img=T.ToTensor()

dtype=torch.float
x=t_img(img).unsqueeze(0)
x=x.to(dtype=dtype,device=device)
model=rNet.cuda(device=device)
model.eval()
score,bbox=model(x)
print(score,bbox)

tensor([[[[ 4.5512e-02,  3.6086e-02, -2.0890e-02,  ...,  8.2953e-02,
            3.0330e-02,  1.0705e-01],
          [ 8.1040e-02,  6.8845e-02,  2.3434e-02,  ...,  9.3288e-02,
            4.9413e-02,  1.8017e-01],
          [ 2.5415e-02,  3.3648e-02,  7.5827e-02,  ..., -5.2495e-02,
           -7.9867e-02,  6.8422e-02],
          ...,
          [-5.9073e-03,  1.0668e-01,  1.0869e-01,  ...,  1.3763e-02,
            6.1012e-02,  4.8112e-02],
          [ 3.5020e-02,  8.8391e-02,  9.6970e-02,  ...,  6.0556e-02,
            1.1276e-01,  4.8211e-02],
          [ 2.4921e-02,  5.5932e-02,  6.7466e-02,  ...,  8.0371e-02,
            8.9775e-02,  6.1158e-02]],

         [[-4.1732e-02, -1.2894e-01, -2.3369e-01,  ..., -1.2007e-02,
            1.6226e-02, -1.2357e-01],
          [-5.7997e-02, -5.3567e-02, -8.0702e-02,  ..., -7.4904e-02,
           -5.4914e-02, -5.0166e-02],
          [-8.1876e-03, -1.4863e-01, -1.2507e-01,  ..., -9.3666e-02,
           -8.7034e-02, -5.7984e-02],
          ...,
     

In [29]:
print(score.shape)
print(bbox.shape)
print(img.shape)

n=1
hh=69
ww=45
n_anchor=9
score=F.softmax(score.view(n, hh, ww, n_anchor, 2),dim=4)

torch.Size([1, 18, 69, 45])
torch.Size([1, 36, 69, 45])
(1113, 720, 3)


In [30]:
np_score=score.squeeze().cpu().detach().numpy()
np_bbox=bbox.squeeze().cpu().detach().numpy()

In [34]:
print(np_score.shape)
np_score[1,:,:]

(69, 45, 9, 2)


array([[[0.5098419 , 0.49015805],
        [0.501039  , 0.498961  ],
        [0.50301576, 0.4969842 ],
        [0.50243145, 0.49756855],
        [0.50827533, 0.49172464],
        [0.4987742 , 0.50122577],
        [0.48763165, 0.5123683 ],
        [0.509349  , 0.49065104],
        [0.49937883, 0.50062114]],

       [[0.5046025 , 0.4953975 ],
        [0.4935676 , 0.5064325 ],
        [0.5086992 , 0.4913008 ],
        [0.46747053, 0.5325295 ],
        [0.46731025, 0.5326897 ],
        [0.5217008 , 0.47829914],
        [0.5009208 , 0.4990792 ],
        [0.50518054, 0.49481946],
        [0.49060622, 0.50939375]],

       [[0.50579894, 0.49420103],
        [0.50967133, 0.49032867],
        [0.49608672, 0.5039133 ],
        [0.49835935, 0.5016407 ],
        [0.50575954, 0.4942404 ],
        [0.50065774, 0.49934223],
        [0.4961901 , 0.50380987],
        [0.50016713, 0.49983287],
        [0.5138465 , 0.48615348]],

       [[0.50972253, 0.49027747],
        [0.4737555 , 0.5262445 ],
        